# REIL Approximate Emulator

I just so happened to stumble upon a little gem: [a working (hopefully) REIL Python translation library](https://github.com/c01db33f/reil). It's completely undocumented and hasn't seen a commit since 2016, but hey, every diamond starts off as coal.

This translator hits some key awesomeness requirements:
 - Translates x86, x86_64, ARMv7 (32-bit), and ARMv8 (64-bit)
 - Implements the [original REIL spec](https://www.zynamics.com/binnavi/manual/html/reil_language.htm), with no fancy/annoying extensions (except for a few genuinely helpful bit-shifting instructions)
 - It's _just_ a translator, with no extra bloat included (CFG generation, etc.)
 - Uses capstone datatypes (which we already use too)

To allow for enhanced documentation and patching, I've forked the translater into [my own repo](https://github.ibm.com/Anthony-Byrne/pyreil) on IBM GitHub.
 
#### Other versions/forks
Looks like a few others have found this gem too.
 - ~~[a slightly newer fork](https://github.com/chubbymaggie/pyreil)~~ not a fork, exactly the same
 - apparently Google made a [C++ version](https://github.com/google/reil) that only supports ARMv8 (?)
 - [a REIL symbolic execution engine](https://github.com/enjhnsn2/reilex) that uses a modified version of pyreil

## Playing with pyreil

In [2]:
# Imports and helpers
import nucleus
from elftools.elf.elffile import ELFFile

def get_raw_bytes(path, start, stop):
    """
    Get the raw bytes between two MEMORY addresses in an ELF binary
    
    :param path: string path to the ELF file
    :param start: the starting address to extract
    :param stop: the last address to extract
    :returns: a raw bytes object
    """
    with open(path, 'rb') as f:
        start_addr = list(ELFFile(f).address_offsets(start))[0]
        f.seek(start_addr)
        return f.read(stop - start)

Let's start with a simple translation of a single basic block. First we'll use nucleus in the usual way to find function boundaries.

In [3]:
BIN_PATH = "/bin/echo"
# Load binary into a nucleus context
nctx = nucleus.load(BIN_PATH)
functs = nctx.cfg.functions

# Print the first few detected functions
print("Found {} functions in {}".format(len(functs), BIN_PATH))
count = 0
for fn in functs[:5]:
    print("{}: Spans 0x{:x} to 0x{:x} with {} basic blocks".format(count, fn.start, fn.end, len(fn.BBs)))
    count += 1

Found 78 functions in /bin/echo
0: Spans 0x1410 to 0x1427 with 3 basic blocks
1: Spans 0x1730 to 0x1736 with 1 basic blocks
2: Spans 0x1c80 to 0x1cb2 with 4 basic blocks
3: Spans 0x1d60 to 0x1de8 with 3 basic blocks
4: Spans 0x1df0 to 0x21e7 with 50 basic blocks


Now let's see the original x86 assembly of one of those BB's

In [4]:
bb = functs[4].BBs[0]
print("The basic block spanning 0x{:x} to 0x{:x} has {} x86 instructions".format(bb.start, bb.end, len(bb.insns)))
for ct, ins in enumerate(bb.insns):
    print("0x{:x}: {} {}".format(ins.start, ins.mnem, ins.op_str))

The basic block spanning 0x1df0 to 0x1e1a has 12 x86 instructions
0x1df0: push r12
0x1df2: push rbp
0x1df3: mov edx, 5
0x1df8: push rbx
0x1df9: mov ebp, edi
0x1dfb: add rsp, -0x80
0x1dff: mov rbx, qword ptr [rip + 0x2062ba]
0x1e06: mov rax, qword ptr fs:[0x28]
0x1e0f: mov qword ptr [rsp + 0x78], rax
0x1e14: xor eax, eax
0x1e16: test edi, edi
0x1e18: je 0x1e48


Now let's see the REIL code

In [5]:
from reil.x86.translator import translate
from time import process_time, monotonic
preload = (process_time(), monotonic())
raw_bytes = get_raw_bytes(BIN_PATH, bb.start, bb.end)
pretrans = (process_time(), monotonic())
reil_bb = list(il_ins for nat_ins in translate(raw_bytes, 0x0, x86_64=True) for il_ins in nat_ins.il_instructions)
fin = (process_time(), monotonic())
print("The REIL translation of that basic block has {} instructions".format(len(reil_bb)))
a = ((pretrans[0] - preload[0]) * 1000, (pretrans[1] - preload[1]) * 1000)
b = ((fin[0] - pretrans[0]) * 1000, (fin[1] - pretrans[1]) * 1000)
print("It took {1}{2:.2f}{0} CPU-ms ({1}{3:.2f}{0} wall-ms) to load raw instructions and {1}{4:.2f}{0} CPU-ms ({1}{5:.2f}{0} wall-ms) to translate them\x1b[0m".format('\u001b[0m', '\u001b[32m', a[0], a[1], b[0], b[1]))
for ins in reil_bb:
    print(str(ins))

The REIL translation of that basic block has 77 instructions
It took 2.88 CPU-ms (2.88 wall-ms) to load raw instructions and 4.33 CPU-ms (4.34 wall-ms) to translate them
sub (rsp, 64), (8, 64), (rsp, 64)
stm (r12, 64), (rsp, 64)
sub (rsp, 64), (8, 64), (rsp, 64)
stm (rbp, 64), (rsp, 64)
str (5, 32), (t00, 64)
str (t00, 64), (rdx, 64)
sub (rsp, 64), (8, 64), (rsp, 64)
stm (rbx, 64), (rsp, 64)
str (rdi, 64), (t00, 32)
str (t00, 32), (t01, 64)
str (t01, 64), (rbp, 64)
sex (-128, 16), (t00, 64)
add (rsp, 64), (t00, 64), (t01, 128)
and (rsp, 64), (9223372036854775808, 64), (t02, 64)
and (t00, 64), (9223372036854775808, 64), (t03, 64)
and (t01, 128), (9223372036854775808, 64), (t04, 64)
xor (t02, 64), (t04, 64), (t05, 64)
xor (t03, 64), (t04, 64), (t06, 64)
and (t05, 64), (t06, 64), (t07, 64)
bisnz (t07, 64), (of, 8)
bisnz (t04, 64), (sf, 8)
and (t01, 128), (18446744073709551615, 64), (t08, 64)
bisz (t08, 64), (zf, 8)
and (t01, 128), (18446744073709551616, 128), (t09, 128)
bisnz (t09, 128), 

It works! Note the notation used for operands here is `([reg name or immediate], [width])`. Also note that you're looking at a string representation of a nicely-designed Python object, so playing with these instructions should be pretty easy.

REIL is clearly very RISC-y, i.e. needs a lot of instructions to do what CISC-y ISAs like x86 can do in one instruction (hence a 77 REIL instruction long translation of 12 x86 instructions. This isn't really an issue for us (we're not storing the instructions, just executing them), but it is a notable observation.

Thankfully, this translator library preserves the relationship between original instrunctions and the REIL instructions they're translated into. We could use this to blacklist particular instructions, or just to collect statistics.

In [6]:
natives = translate(raw_bytes, 0x0, x86_64=True)
for nat_ins in natives:
    print("0x{:02x}: {}\t[{} REILs]".format(nat_ins.address, nat_ins.mnemonic, len(nat_ins.il_instructions)))
    for reil_ins in nat_ins.il_instructions:
        print("   = " + str(reil_ins))

0x00: push r12	[2 REILs]
   = sub (rsp, 64), (8, 64), (rsp, 64)
   = stm (r12, 64), (rsp, 64)
0x02: push rbp	[2 REILs]
   = sub (rsp, 64), (8, 64), (rsp, 64)
   = stm (rbp, 64), (rsp, 64)
0x03: mov edx, 5	[2 REILs]
   = str (5, 32), (t00, 64)
   = str (t00, 64), (rdx, 64)
0x08: push rbx	[2 REILs]
   = sub (rsp, 64), (8, 64), (rsp, 64)
   = stm (rbx, 64), (rsp, 64)
0x09: mov ebp, edi	[3 REILs]
   = str (rdi, 64), (t00, 32)
   = str (t00, 32), (t01, 64)
   = str (t01, 64), (rbp, 64)
0x0b: add rsp, -0x80	[22 REILs]
   = sex (-128, 16), (t00, 64)
   = add (rsp, 64), (t00, 64), (t01, 128)
   = and (rsp, 64), (9223372036854775808, 64), (t02, 64)
   = and (t00, 64), (9223372036854775808, 64), (t03, 64)
   = and (t01, 128), (9223372036854775808, 64), (t04, 64)
   = xor (t02, 64), (t04, 64), (t05, 64)
   = xor (t03, 64), (t04, 64), (t06, 64)
   = and (t05, 64), (t06, 64), (t07, 64)
   = bisnz (t07, 64), (of, 8)
   = bisnz (t04, 64), (sf, 8)
   = and (t01, 128), (18446744073709551615, 64), (t08,

## Emulator code

**NOTE: This is the original version of the emulator code, preserved for posterity. The most recent version can be found in `util/acevm.py`**

In [7]:
from enum import Enum
class REILRegContext(Enum):
    """
    Enumerated type representing different possible startup register contexts
    """
    # Undefined/custom (used when user provides context)
    undefined = 0
    # All registers = 0 
    zeros = 1
    # All registers = 1
    ones = 2
    # All registers = 1,073,741,823 (0x3FFFFFFF, or half of the 32-bit signed max)
    halfmax_signed = 3
    # All registers = 2,147,483,647 (0x7FFFFFFF, or half of the 32-bit unsigned max)
    halfmax_unsigned = 4
    # All registers = -1,073,741,823 (0xC0000001, or half of the 32-bit signed min)
    halfmin_signed = 5
    # All registers = 0x55555555 (0b01010101010101010101010101010101)
    bitweave_one = 6
    # All registers = 0x0F0F0F0F (0b00001111000011110000111100001111)
    bitweave_four = 7
    # Register values count up from zero
    countup = 8
    # Register values count down from 31
    countdown = 9
    # Random permutations (see implementation)
    random1 = 10
    random2 = 11
    random3 = 12
    random4 = 13
    
class REILMemContext(Enum):
    """
    Enumerated type representing different possible startup memory contexts
    """
    # Undefined/custom (used when user provides context)
    undefined = 0
    # All addresses = 0
    zeros = 1
    # ...
    # TODO: determine if other memory contexts are helpful
    # ...
    # All memory cells have their address as their value
    address = 2
    

In [14]:
import random
from reil.definitions import ImmediateOperand, TemporaryOperand, RegisterOperand, _opcode_to_string

class REILApproximateVM():
    """
    "Approximate" virtual machine for REIL. Designed for fast collection of context changes, not correct execution
    """
    def __init__(self, reg_context, mem_context, dynamic_mem = True):
        """
        Constructor
        """
        # Memory setup
        self.mem_context = mem_context
        self.dynamic_mem = dynamic_mem
        self.memory = {}
        
        # Register setup
        X86_REG_NAMES = ['al', 'ah', 'bl', 'bh', 'cl', 'ch', 'dl', 'dh', 'sil', 'dil', 'bpl', 'spl', 'r8b', 'r9b', 'r10b', 'r11b', 'r12b', 'r13b', 'r14b', 'r15b', 'ax', 'bx', 'cx', 'dx', 'si', 'di', 'bp', 'sp', 'r8w', 'r9w', 'r10w', 'r11w', 'r12w', 'r13w', 'r14w', 'r15w', 'eax', 'ebx', 'ecx', 'edx', 'esi', 'edi', 'ebp', 'esp', 'r8d', 'r9d', 'r10d', 'r11d', 'r12d', 'r13d', 'r14d', 'r15d', 'rax', 'rbx', 'rcx', 'rdx', 'rsi', 'rdi', 'rbp', 'rsp', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'rip']
        NUM_T_REGS = 32
        NUM_N_REGS = len(X86_REG_NAMES)
        self.reg_context = reg_context
        n_regs = {}
        
        
        if not isinstance(reg_context, REILRegContext) and hasattr(reg_context, "__getitem__"):
            # Allows user to provide array of registers for custom contexts
            self.reg_context = REILRegContext.undefined
            self.t_regs = reg_context
            self.n_regs = dict(zip(X86_REG_NAMES, reg_context))
        elif reg_context == REILRegContext.zeros:
            self.t_regs = [0] * NUM_T_REGS
            self.n_regs = dict(zip(X86_REG_NAMES, [0] * NUM_N_REGS))
        elif reg_context == REILRegContext.ones:
            self.t_regs = [1] * NUM_T_REGS
            self.n_regs = dict(zip(X86_REG_NAMES, [1] * NUM_N_REGS))
        elif reg_context == REILRegContext.halfmax_signed:
            self.t_regs = [0x3FFFFFFF] * NUM_T_REGS
            self.n_regs = dict(zip(X86_REG_NAMES, [0x3FFFFFFF] * NUM_N_REGS))
        elif reg_context == REILRegContext.halfmax_unsigned:
            self.t_regs = [0x7FFFFFFF] * NUM_T_REGS
            self.n_regs = dict(zip(X86_REG_NAMES, [0x7FFFFFFF] * NUM_N_REGS))
        elif reg_context == REILRegContext.halfmin_signed:
            self.t_regs = [0xC0000001] * NUM_T_REGS
            self.n_regs = dict(zip(X86_REG_NAMES, [0xC0000001] * NUM_N_REGS))
        elif reg_context == REILRegContext.bitweave_one:
            self.t_regs = [0x55555555] * NUM_T_REGS
            self.n_regs = dict(zip(X86_REG_NAMES, [0x55555555] * NUM_N_REGS))
        elif reg_context == REILRegContext.bitweave_four:
            self.t_regs = [0x0F0F0F0F] * NUM_T_REGS
            self.n_regs = dict(zip(X86_REG_NAMES, [0x0F0F0F0F] * NUM_N_REGS))
        elif reg_context == REILRegContext.countup:
            self.t_regs = list(range(NUM_T_REGS))
            self.n_regs = dict(zip(X86_REG_NAMES, list(range(NUM_N_REGS))))
        elif reg_context == REILRegContext.countdown:
            self.t_regs = list(range(NUM_T_REGS-1, -1, -1))
            self.n_regs = dict(zip(X86_REG_NAMES, list(range(NUM_N_REGS-1, -1, -1))))
        elif reg_context == REILRegContext.random1:
            self.t_regs = REILApproximateVM.__random_context(NUM_T_REGS, 100)
            self.n_regs = dict(zip(X86_REG_NAMES, REILApproximateVM.__random_context(NUM_N_REGS, 101)))
        elif reg_context == REILRegContext.random2:
            self.t_regs = REILApproximateVM.__random_context(NUM_T_REGS, 200)
            self.n_regs = dict(zip(X86_REG_NAMES, REILApproximateVM.__random_context(NUM_N_REGS, 201)))
        elif reg_context == REILRegContext.random3:
            self.t_regs = REILApproximateVM.__random_context(NUM_T_REGS, 300)
            self.n_regs = dict(zip(X86_REG_NAMES, REILApproximateVM.__random_context(NUM_N_REGS, 301)))
        elif reg_context == REILRegContext.random4:
            self.t_regs = REILApproximateVM.__random_context(NUM_T_REGS, 400)
            self.n_regs = dict(zip(X86_REG_NAMES, REILApproximateVM.__random_context(NUM_N_REGS, 401)))
        else:
            raise ValueError("Invalid register context")
    
    @classmethod
    def __random_context(cls, num_regs, seed):
        """
        Deterministic random context generator
        """
        t_regs = [0] * num_regs
        random.seed(seed)
        for s in range(num_regs):
            t_regs[s] = random.randint(-1073741823, 1073741823)
        return t_regs
    
    def __set_reg(self, value, reg):
        """
        Stores a value in a register
        """
        if isinstance(reg, TemporaryOperand):
            self.t_regs[int(reg.name[1:])] = value
        elif isinstance(reg, RegisterOperand):
            self.n_regs[reg.name] = value
            pass
        else:
            raise ValueError("Invalid register provided")
            
    def __resolve_op(self, operand):
        """
        Given an operand, return its value
        """
        if isinstance(operand, ImmediateOperand):
            return operand.value
        elif isinstance(operand, TemporaryOperand):
            return self.t_regs[int(operand.name[1:])]
        elif isinstance(operand, RegisterOperand):
            return self.n_regs[reg.name]
        else:
            raise ValueError("Invalid register provided")
        
    #### BEGIN INSTRUCTION IMPLEMENTATION ####
    def __add(self, input0, input1, output):
        """
        Adds the two values given in the first and second operand and writes the result to the third operand. 
        The input operands can be literals and register values. The output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) + self.__resolve_op(input1), output)
        return
    
    def __and(self, input0, input1, output):
        """
        Binary AND operation that connects the first two operands and stores the result in the third operand. 
        The input operands can be literals and register values. The output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) & self.__resolve_op(input1), output)
        return
    
    def __bisz(self, input0, input1, output):
        """
        Sets a flag depending on whether another value is zero. 
        The input operand can be a literal or a register value. The output operand is a register.
        """
        self.__set_reg(1 if self.__resolve_op(input0) == 0 else 0, output)
        return
        
    def __bsh(self, input0, input1, output):
        """
        Performs a logical shift on a value. If the second operand is positive, the shift is a left-shift. 
        If the second operand is negative, the shift is a right-shift. The two input operands can be either 
        registers or literals while the output operand must be a register.
        """
        if self.__resolve_op(input1) > 0:
            self.__set_reg(self.__resolve_op(input0) << self.__resolve_op(input1), output)
        else:
            self.__set_reg(self.__resolve_op(input0) >> self.__resolve_op(input1), output)
        return
    
    def __div(self, input0, input1, output):
        """
        Performs an unsigned division on the two input operands. The first input operand is the dividend, 
        the second input operand is the divisor. The two input operands can be either registers or literals 
        while the output operand must be a register.
        
        Imp. Note: output is zero on divide-by-zero
        """
        try:
            self.__set_reg(self.__resolve_op(input0) // self.__resolve_op(input1), output)
        except ZeroDivisionError:
            self.__set_reg(0, output)
        return
    
    def __jcc(self, input0, input1, output):
        """
        Conditional jump. Not implemented.
        """
        pass
    
    def __ldm(self, input0, input1, output):
        """
        Loads a value from memory. The first operand specifies the address to read from. It can be either 
        a register or a literal. The third operand must be a register where the loaded value is stored. 
        The size of the third operand determines how many bytes are read from memory.
        
        Imp. Note: we only read the address specifed, nothing beyond that
        """
        # If dynamic memory is on, we check if that address has been written to yet
        if self.dynamic_mem:
            saddr = str(self.__resolve_op(input0))
            try:
                self.__set_reg(self.memory[saddr], output)
            except KeyError:
                # if it hasn't been written to, we just return the address as the value
                # TODO: IMPLEMENT OTHER MemContexts
                self.__set_reg(self.__resolve_op(input0), output)
        else:
            # if dynamic memory is off, we just return the address as the value
            self.__set_reg(self.__resolve_op(input0), output)
        return
    
    def __mod(self, input0, input1, output):
        """
        Performs a modulo operation on the first two operands. The two input operands can be either registers 
        or literals while the output operand must be a register.
        
        Imp. Note: output is zero on divide-by-zero
        """
        try:
            self.__set_reg(self.__resolve_op(input0) % self.__resolve_op(input1), output)
        except ZeroDivisionError:
            self.__set_reg(0, output)
        return
    
    def __mul(self, input0, input1, output):
        """
        Performs an unsigned multiplication on the two input operands. The two input operands can be either 
        registers or literals while the output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) * self.__resolve_op(input1), output)
        return
        
    def __nop(self, input0, input1, output):
        """
        Does nothing.
        """
        pass
    
    def __or(self, input0, input1, output):
        """
        Binary OR operation that connects the first two operands and stores the result in the third operand. 
        The input operands can be literals and register values. The output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) | self.__resolve_op(input1), output)
        return
    
    def __stm(self, input0, input1, output):
        """
        Stores a value to memory. The first operand is the register value or literal to be stored in memory. 
        The third operand is the register value or literal that contains the memory address where the value is stored. 
        The size of the first operand determines the number of bytes to be written to memory.
        """
        # Only does anything if dynamic memory is on
        if self.dynamic_mem:
            self.memory[str(self.__resolve_op(output))] = self.__resolve_op(input0)
        return
    
    def __str(self, input0, input1, output):
        """
        Copies a value to a register. The input operand can be either a literal or a register. The output operand must 
        be a register. If the output operand is of a larger size than the input operand, the input is zero-extended.
        """
        self.__set_reg(self.__resolve_op(input0), output)
        return
    
    def __sex(self, input0, input1, output):
        """
        Functionally identical to STR in this implementation
        """
        self.__set_reg(self.__resolve_op(input0), output)
        return
    
    def __sub(self, input0, input1, output):
        """
        Subtracts the second input operand from the first input operand and writes the result to the output operand.
        The input operands can be literals and register values. The output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) - self.__resolve_op(input1), output)
        return 
    
    def __undef(self, input0, input1, output):
        """
        Flags a register value as undefined. Not implemented.
        """
        pass
    
    def __unkn(self, input0, input1, output):
        """
        Untranslatable native instruction placeholder. Does nothing.
        """
        pass
    
    def __xor(self, input0, input1, output):
        """
        Binary XOR operation that connects the first two operands and stores the result in the third operand. 
        The input operands can be literals and register values. The output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) ^ self.__resolve_op(input1), output)
        return 
    
    def __bisnz(self, input0, input1, output):
        """
        Sets a flag depending on whether another value is nonzero. The input operand can be a literal or 
        a register value. The output operand is a register.
        """
        # Assuming flag = 1 if nonzero
        self.__set_reg(0 if self.__resolve_op(input0) == 0 else 1, output)
        return
    
    def __equ(self, input0, input1, output):
        """
        Sets a flag depending on whether another two values are equal. The input operands can be literal or 
        register values. The output operand is a register.
        """
        # Assuming flag = 1 if equal
        self.__set_reg(1 if self.__resolve_op(input0) == self.__resolve_op(input1) else 0, output)
        return
    
    def __lshl(self, input0, input1, output):
        """
        Performs a logical left shift on a value. The two input operands can be either registers or literals 
        while the output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) << self.__resolve_op(input1), output)
        return
    
    def __lshr(self, input0, input1, output):
        """
        Performs a logical right shift on a value. The two input operands can be either registers or literals 
        while the output operand must be a register.
        """
        self.__set_reg(self.__resolve_op(input0) >> self.__resolve_op(input1), output)
        return
    
    def __ashr(self, input0, input1, output):
        #TODO implement?
        self.__lshr(self, input0, input1, output)
        return
    
    #### END INSTRUCTION IMPLEMENTATION ####
    
    def execute(self, instruction):
        """
        Takes in a single REIL instruction and executes it in the approximate virtual machine
        """
        op = getattr(self, "_REILApproximateVM__" + _opcode_to_string(instruction.opcode))
        op(instruction.input0, instruction.input1, instruction.output)
        #print(op)
        return


## Emulator Testing

We'll now try running a simple program that stores two numbers in temp registers and adds them to a third.

In [9]:
from reil.definitions import Instruction, ADD, SUB, STR
# Test some instructions
instructions = [];
instructions.append(Instruction(STR, ImmediateOperand(33, 32), None, TemporaryOperand(1, 32)))
instructions.append(Instruction(STR, ImmediateOperand(3, 32), None, TemporaryOperand(2, 32)))
instructions.append(Instruction(ADD, TemporaryOperand(1, 32), TemporaryOperand(2, 32), TemporaryOperand(3, 32)))

# Setup the VM
ravm = REILApproximateVM(REILRegContext.zeros, REILMemContext.address)

for ins in instructions:
    print(ravm.t_regs)
    print(ins)
    ravm.execute(ins)
    
print(ravm.t_regs)

assert(ravm.t_regs[1:4] == [33, 3, 36])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
str (33, 32), (t01, 32)
[0, 33, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
str (3, 32), (t02, 32)
[0, 33, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
add (t01, 32), (t02, 32), (t03, 32)
[0, 33, 3, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Now we'll try running the example basic block that we translated earlier

In [20]:
# Setup the VM
ravm = REILApproximateVM(REILRegContext.countup, REILMemContext.address)

for ins in reil_bb:
    #print(ravm.t_regs)
    print(ins)
    ravm.execute(ins)
    
print(ravm.t_regs)

sub (rsp, 64), (8, 64), (rsp, 64)
stm (r12, 64), (rsp, 64)
sub (rsp, 64), (8, 64), (rsp, 64)
stm (rbp, 64), (rsp, 64)
str (5, 32), (t00, 64)
str (t00, 64), (rdx, 64)
sub (rsp, 64), (8, 64), (rsp, 64)
stm (rbx, 64), (rsp, 64)
str (rdi, 64), (t00, 32)
str (t00, 32), (t01, 64)
str (t01, 64), (rbp, 64)
sex (-128, 16), (t00, 64)
add (rsp, 64), (t00, 64), (t01, 128)
and (rsp, 64), (9223372036854775808, 64), (t02, 64)
and (t00, 64), (9223372036854775808, 64), (t03, 64)
and (t01, 128), (9223372036854775808, 64), (t04, 64)
xor (t02, 64), (t04, 64), (t05, 64)
xor (t03, 64), (t04, 64), (t06, 64)
and (t05, 64), (t06, 64), (t07, 64)
bisnz (t07, 64), (of, 8)
bisnz (t04, 64), (sf, 8)
and (t01, 128), (18446744073709551615, 64), (t08, 64)
bisz (t08, 64), (zf, 8)
and (t01, 128), (18446744073709551616, 128), (t09, 128)
bisnz (t09, 128), (cf, 8)
str (t01, 128), (t10, 8)
lshr (t10, 8), (4, 8), (t11, 8)
xor (t10, 8), (t11, 8), (t10, 8)
and (t10, 8), (15, 8), (t11, 8)
lshr (38505, 16), (t11, 8), (t12, 16)
an